## Notebook Setup 
The following cell will install Drake, checkout the underactuated repository, and set up the path (only if necessary).
- On Google's Colaboratory, this **will take approximately two minutes** on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours.  Colab will ask you to "Reset all runtimes"; say no to save yourself the reinstall.
- On Binder, the machines should already be provisioned by the time you can run this; it should return (almost) instantly.

More details are available [here](http://underactuated.mit.edu/underactuated.html?chapter=drake).

In [ ]:
try:
  import pydrake
  import underactuated
except ImportError:
  !curl -s https://raw.githubusercontent.com/RussTedrake/underactuated/master/scripts/setup/jupyter_setup.py > jupyter_setup.py
  from jupyter_setup import setup_underactuated
  setup_underactuated()

# Setup matplotlib backend (to notebook, if possible, or inline).  
from underactuated.jupyter import setup_matplotlib_backend
plt_is_interactive = setup_matplotlib_backend()


## Balancing around the upright using LQR


In [ ]:
from IPython import get_ipython
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import numpy as np

from pydrake.all import (AddMultibodyPlantSceneGraph, DiagramBuilder, 
                         LinearQuadraticRegulator, Parser, PlanarSceneGraphVisualizer, 
                         SceneGraph, Simulator)
from underactuated import FindResource


def UprightState():
    state = (0, np.pi, 0, 0)
    return state


def BalancingLQR(plant):
    # Design an LQR controller for stabilizing the CartPole around the upright.
    # Returns a (static) AffineSystem that implements the controller (in
    # the original CartPole coordinates).

    context = plant.CreateDefaultContext()
    plant.get_actuation_input_port().FixValue(context, [0])

    context.get_mutable_continuous_state_vector().SetFromVector(UprightState())

    Q = np.diag((10., 10., 1., 1.))
    R = [1]

    # MultibodyPlant has many (optional) input ports, so we must pass the
    # input_port_index to LQR.
    return LinearQuadraticRegulator(
        plant,
        context,
        Q,
        R,
        input_port_index=plant.get_actuation_input_port().get_index())


builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
file_name = FindResource("models/cartpole.urdf")
Parser(plant).AddModelFromFile(file_name)
plant.Finalize()

controller = builder.AddSystem(BalancingLQR(plant))
builder.Connect(plant.get_state_output_port(), controller.get_input_port(0))
builder.Connect(controller.get_output_port(0),
                plant.get_actuation_input_port())

# Setup visualization
visualizer = builder.AddSystem(
    PlanarSceneGraphVisualizer(scene_graph, xlim=[-2.5, 2.5], ylim=[-1, 2.5], show=plt_is_interactive))
builder.Connect(scene_graph.get_pose_bundle_output_port(),
                visualizer.get_input_port(0))

diagram = builder.Build()

# Set up a simulator to run this diagram
simulator = Simulator(diagram)
if plt_is_interactive:
    simulator.set_target_realtime_rate(1.)
context = simulator.get_mutable_context()


In [ ]:
# Simulate
if not plt_is_interactive:
    visualizer.start_recording()
duration = 8.0 if get_ipython() else 0.1 # sets a shorter duration during testing

for i in range(5):
    context.SetTime(0.)
    context.SetContinuousState(UprightState() + 0.1 * np.random.randn(4,))
    simulator.Initialize()
    simulator.AdvanceTo(duration)

if not plt_is_interactive:
    ani = visualizer.get_recording_as_animation()
    HTML(ani.to_jshtml())    
